Let's say that we've gotten a dataset that we'd like to benchmark for active learning. 

In [1]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=100_000, n_classes=4, n_informative=10)

We will use modAL to handle some of the heavy liften. That means that we also need to write a compatible sampler if we want to have the random benchmark around.

In [2]:
import numpy as np
from modAL.uncertainty import entropy_sampling
from sklearn.linear_model import LogisticRegression

def randomly(classifier, X, n_instances=1):
    idx =  np.random.randint(0, X.shape[0] - 1, n_instances)
    return idx, X

We can now use the `run_experiment` from the `skteach.py` script to automate the data collection.

In [3]:
from skteach import run_experiment

In [6]:
?run_experiment

Signature:
run_experiment(
    name,
    data,
    estimator,
    strategy,
    batch_size=100,
    n_batch=10,
    start_size=50,
    idx_start=None,
)
Docstring: <no docstring>
File:      ~/Development/scikit-teach/skteach.py
Type:      function


In [4]:
data = []
for i in range(5): 
    data += run_experiment("entropy", data=(X, y), estimator=LogisticRegression(), strategy=entropy_sampling, n_batch=50)
for i in range(5):
    data += run_experiment("randomly", data=(X, y), estimator=LogisticRegression(), strategy=randomly, n_batch=50)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 15.17it/s]


We can compare the scores with an altair chart now.

In [5]:
import pandas as pd
import altair as alt

pltr = pd.DataFrame(data)

(alt.Chart(pltr)
  .mark_line()
  .encode(x='batch', y='score', color='name', detail='run_id')
  .properties(width=600, height=250)
  .interactive())

alt.Chart(...)

Lesson one in active learning: random sampling is a suprisingly strong benchmark